In [579]:
import rasterio
import rioxarray
import os
import numpy as np

In [580]:
tif_pre_fire = "../rasters/lansat/2023_valpo_swir22-nir-red.tif"
tif_post_fire = "../rasters/lansat/2024_valpo_swir22-nir-red.tif"

pre_fire = rasterio.open(tif_pre_fire)
post_fire = rasterio.open(tif_post_fire)

In [581]:
# Usamos las bandas red y nir para calcular el NBR
nbr_pre_fire = (pre_fire.read(3) - pre_fire.read(2)) / (pre_fire.read(3) + pre_fire.read(2))
nbr_post_fire = (post_fire.read(3) - post_fire.read(2)) / (post_fire.read(3) + post_fire.read(2))

# Con estos calculamos el dNBR
dnbr = nbr_pre_fire - nbr_post_fire

# Guardamos el resultado en un nuevo archivo
profile = pre_fire.profile
profile.update(dtype=rasterio.float32, count=1)
# with rasterio.open("../rasters/lansat/dnbr_2023_2024.tif", "w", **profile) as dst:
#     dst.write(dnbr, 1)
    

In [582]:
# Ahora creamos categorias de valores para dNBR
dnbr_ranges = {
    'enhaced_regrowth_high': (-0.500, -0.251),
    'enhaced_regrowth_low': (-0.250, -0.101),
    'unburned': (-0.100, 0.99),
    'low_severity': (0.100, 0.269),
    'moderate_low_severity': (0.270, 0.439),
    'moderate_high_severity': (0.440, 0.659),
    'high_severity': (0.660, 1.300)
}

dnbr_gray_values = {
    'enhaced_regrowth_high': 0,
    'enhaced_regrowth_low': 1,
    'unburned': 2,
    'low_severity': 3,
    'moderate_low_severity': 5,
    'moderate_high_severity': 6,
    'high_severity': 7
}

dnbr_counts = {
    'enhaced_regrowth_high': 0,
    'enhaced_regrowth_low': 0,
    'unburned': 0,
    'low_severity': 0,
    'moderate_low_severity': 0,
    'moderate_high_severity': 0,
    'high_severity': 0
}

In [583]:
# Recorremos la mascara dNBR y contamos cuantos pixeles hay en cada categoria
for key, value in dnbr_ranges.items():
    dnbr_counts[key] = ((dnbr >= value[0]) & (dnbr <= value[1])).sum()
    
dnbr_counts

{'enhaced_regrowth_high': 4879,
 'enhaced_regrowth_low': 110224,
 'unburned': 2507095,
 'low_severity': 5314,
 'moderate_low_severity': 613,
 'moderate_high_severity': 78,
 'high_severity': 109}

In [584]:
# Multiplicamos por 30x30 para obtener el area en metros cuadrados

# Cambiamos el tipo de dato de dnbr_counts a int64
dnbr_counts = {key: int(value) * 30 * 30 for key, value in dnbr_counts.items()}

# Multiplicamos por 30x30 para obtener el area en metros cuadrados
dnbr_counts = {key: value * 30 * 30 for key, value in dnbr_counts.items()}

dnbr_counts

{'enhaced_regrowth_high': 4391100,
 'enhaced_regrowth_low': 99201600,
 'unburned': 2256385500,
 'low_severity': 4782600,
 'moderate_low_severity': 551700,
 'moderate_high_severity': 70200,
 'high_severity': 98100}

In [585]:
# Ahora creamos un nuevo tif 
# Los valores de la mascara seran los valores de dnbr_gray_values
# con los valores de dnbr_counts
dnbr_mask = dnbr.copy()
for key, value in dnbr_ranges.items():
    dnbr_mask[(dnbr >= value[0]) & (dnbr <= value[1])] = dnbr_gray_values[key]
    
dnbr_mask[np.logical_not(dnbr_mask> 0)] = 0
    
profile = pre_fire.profile
profile.update(dtype=rasterio.float32, count=1)

path_dnbr_2023_2024_discretised = "../rasters/lansat/dnbr_2023_2024_discretised.tif"
if os.path.exists(path_dnbr_2023_2024_discretised):
    os.remove(path_dnbr_2023_2024_discretised)
with rasterio.open("../rasters/lansat/dnbr_2023_2024_discretised.tif", "w", **profile) as dst:
    dst.write(dnbr_mask, 1)

In [586]:
import numpy as np

# Obtener el rango de valores de dNBR
min_value = np.nanmin(dnbr_mask)
max_value = np.nanmax(dnbr_mask)

print('Valor mínimo de dNBR:', min_value)
print('Valor máximo de dNBR:', max_value)

Valor mínimo de dNBR: 0.0
Valor máximo de dNBR: 7.0


In [587]:
np.unique(dnbr_mask.flatten())

array([0., 1., 2., 3., 5., 6., 7.], dtype=float32)

In [588]:
dnbr_mask

array([[0., 0., 0., ..., 2., 2., 2.],
       [0., 0., 0., ..., 2., 2., 2.],
       [0., 0., 0., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]], dtype=float32)